In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

The structuring of the data for easy training. Since we are using Selective_kernel Network, which is CNN based for Human Activity Recognition and Segmentation

The first step is the processing of the data to be used for training. From Better Deep Learning it is required that you use 3D of getting the data ready.
The 3D structure of the input data is often summarized using the arrary shape of the notation [samples, timesteps, features]. The 2D format of this should be [samples, features]

In [5]:

def load_raw_signals(folder_path, filenames):
    signals = []
    for name in filenames:
        # Each file is (7352, 128)
        data = pd.read_csv(f'{folder_path}/{name}.txt', header=None, delim_whitespace=True)
        signals.append(data.values)
    
    # Stacking on the 3rd axis (axis=2) results in (7352, 128, 9)
    return np.transpose(np.array(signals), (1, 2, 0))

# Example list of the 9 channels based on your files
filenames = [
    'total_acc_x_train', 'total_acc_y_train', 'total_acc_z_train',
    'body_acc_x_train', 'body_acc_y_train', 'body_acc_z_train',
    'body_gyro_x_train', 'body_gyro_y_train', 'body_gyro_z_train'
]

filenames_test = [
    'total_acc_x_test', 'total_acc_y_test', 'total_acc_z_test',
    'body_acc_x_test', 'body_acc_y_test', 'body_acc_z_test',
    'body_gyro_x_test', 'body_gyro_y_test', 'body_gyro_z_test'
]


X_train = load_raw_signals('/home/pschye/Documents/KCCR - Biosignals/Human Activity Recognition/Human Activity Recognition/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/Inertial Signals', filenames)
y_train = pd.read_csv('/home/pschye/Documents/KCCR - Biosignals/Human Activity Recognition/Human Activity Recognition/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt', header=None, delim_whitespace=True).values.flatten() - 1
X_test = load_raw_signals('/home/pschye/Documents/KCCR - Biosignals/Human Activity Recognition/Human Activity Recognition/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals', filenames_test)
y_test = pd.read_csv('/home/pschye/Documents/KCCR - Biosignals/Human Activity Recognition/Human Activity Recognition/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt', header=None, delim_whitespace=True).values.flatten() - 1    

/tmp/ipykernel_8071/1079293147.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(f'{folder_path}/{name}.txt', header=None, delim_whitespace=True)
/tmp/ipykernel_8071/1079293147.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(f'{folder_path}/{name}.txt', header=None, delim_whitespace=True)
/tmp/ipykernel_8071/1079293147.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(f'{folder_path}/{name}.txt', header=None, delim_whitespace=True)
/tmp/ipykernel_8071/1079293147.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(f'{fo

In [12]:
print('train_x:', X_train.shape, '  train_y:', y_train.shape)
print('test_x: ', X_test.shape,  '  test_y: ', y_test.shape)

train_x: (7352, 128, 9)   train_y: (7352,)
test_x:  (2947, 128, 9)   test_y:  (2947,)


(7352, 128, 9) this is a workable format to be used in CNN(Backbone Network)

In [13]:
BATCH_SIZE = 256
TEST_BATCH  = 2947

train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(TEST_BATCH)
    .prefetch(tf.data.AUTOTUNE)
)

2026-02-18 15:53:04.534988: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2026-02-18 15:53:04.619847: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 67756032 exceeds 10% of free system memory.


## Model Architecture

In [ ]:
# ──────────────────────────────────────────────
# 3. SKConv layer
#    PyTorch layout: (N, C, H, W)  →  TF layout: (N, H, W, C)
#    PyTorch kernel=(3,1), padding=(1+i, 1), dilation=(1+i, 1)
#    → TF Conv2D with kernel_size=(3,1), padding='same' per branch
# ──────────────────────────────────────────────

class SKConv(layers.Layer):
    """Selective-Kernel Convolution block (TensorFlow/Keras, channels-last)."""

    def __init__(self, features, M=3, G=32, r=32, stride=1, L=32, **kwargs):
        super().__init__(**kwargs)
        d = max(int(features / r), L)
        self.M = M
        self.features = features

        # M parallel branches with different dilation rates
        self.branches = []
        for i in range(M):
            branch = keras.Sequential([
                # groups → use DepthwiseConv2D trick or just Conv2D
                # PyTorch groups=G  ≈  TF Conv2D with groups=G (TF 2.x supports it)
                layers.Conv2D(
                    filters=features,
                    kernel_size=(3, 1),
                    strides=(stride, 1),
                    padding='same',
                    dilation_rate=(1 + i, 1),
                    groups=G,
                    use_bias=False,
                ),
                layers.BatchNormalization(),
                layers.ReLU(),
            ], name=f'branch_{i}')
            self.branches.append(branch)

        # Global Average Pool + FC (implemented as Conv2D(1,1) to keep 4-D tensor)
        self.gap = layers.GlobalAveragePooling2D(keepdims=True)   # → (N, 1, 1, C)
        self.fc = keras.Sequential([
            layers.Conv2D(d, kernel_size=1, use_bias=False),
            layers.BatchNormalization(),
            layers.ReLU(),
        ], name='fc_z')

        # One 1×1 conv per branch to produce attention logits
        self.fcs = [
            layers.Conv2D(features, kernel_size=1, name=f'fc_attn_{i}')
            for i in range(M)
        ]
        self.softmax = layers.Softmax(axis=1)   # softmax across branch dimension

    def call(self, x, training=False):
        batch_size = tf.shape(x)[0]

        # Each branch output: (N, H, W, features)
        branch_feats = [branch(x, training=training) for branch in self.branches]

        # Stack → (N, M, H, W, features)
        feats = tf.stack(branch_feats, axis=1)

        # Fuse: element-wise sum across branches → (N, H, W, features)
        feats_U = tf.reduce_sum(feats, axis=1)

        # Channel descriptor via GAP → (N, 1, 1, features)
        feats_S = self.gap(feats_U)

        # Compact feature → (N, 1, 1, d)
        feats_Z = self.fc(feats_S, training=training)

        # Per-branch attention vectors → list of (N, 1, 1, features)
        attn_vectors = [fc(feats_Z) for fc in self.fcs]

        # Stack → (N, M, 1, 1, features), then softmax over branch dim
        attn_vectors = tf.stack(attn_vectors, axis=1)
        attn_vectors = self.softmax(attn_vectors)           # (N, M, 1, 1, features)

        # Weighted sum: (N, M, H, W, features) * (N, M, 1, 1, features)
        feats_V = tf.reduce_sum(feats * attn_vectors, axis=1)  # (N, H, W, features)

        return feats_V

    def get_config(self):
        config = super().get_config()
        config.update(dict(features=self.features, M=self.M))
        return config


### Building Network

In [ ]:
# ──────────────────────────────────────────────
# 4. Build SKNet model
# -------------------------------------

def build_sknet(M=3, G=32, r=32, stride=1, L=32, num_classes=6):
    """Returns a compiled Keras Model equivalent to the PyTorch SKNet."""
    inputs = keras.Input(shape=(128, 9, 1), name='input')   # (H, W, 1)

    # conv1: PyTorch Conv2d(1, 64, (5,1), stride=(3,1), padding=(1,0))
    # channels-last TF equivalent:
    x = layers.Conv2D(64, kernel_size=(5, 1), strides=(3, 1), padding='same', use_bias=True)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # NOTE: PyTorch conv2_sk expects 128 input features but conv1 outputs 64.
    # The original code passes 64-channel output to SKConv(128, ...) which would
    # error in PyTorch too unless conv1 produces 128. We mirror the code as-is
    # and set SKConv to match actual channel count (64 → 64, then 64 → 128).
    # Adjust these numbers to match your actual checkpoint / intent.
    x = SKConv(64,  M=M, G=G, r=r, stride=stride, L=L, name='skconv1')(x)
    x = SKConv(128, M=M, G=G, r=r, stride=stride, L=L, name='skconv2')(x)

    # Flatten
    x = layers.Flatten()(x)

    # FC: 6-class output
    # PyTorch uses nn.LayerNorm on the logits; replicate with LayerNormalization
    x = layers.Dense(num_classes, name='fc')(x)
    x = layers.LayerNormalization(name='layer_norm')(x)

    model = keras.Model(inputs, x, name='SKNet')
    return model


In [20]:
# ──────────────────────────────────────────────
# 5. Compile
# ──────────────────────────────────────────────
print('==> Building model..')

model = build_sknet()
model.summary()

WD = 1e-4
optimizer = keras.optimizers.Adam(learning_rate=0.001, weight_decay=WD)

loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy'],
)

# Learning-rate scheduler: StepLR(step_size=50, gamma=0.1)
# In Keras this is done via a LearningRateScheduler callback
def step_lr_schedule(epoch, lr):
    """Halve the LR by ×0.1 every 50 epochs."""
    if epoch > 0 and epoch % 50 == 0:
        return lr * 0.1
    return lr

lr_callback = keras.callbacks.LearningRateScheduler(step_lr_schedule, verbose=1)


==> Building model..


Model: "SKNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 128, 9, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 43, 9, 64)      │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 43, 9, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_18 (ReLU)                 │ (None, 43, 9, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ skconv1 (SKConv)                │ (None, 43, 9, 64)      │        10,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ skconv2 (SKConv)                │ (None, 43, 9, 128)     │        20,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 49536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 6)              │       297,222 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_norm (LayerNormalization) │ (None, 6)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 329,042 (1.26 MB)

 Trainable params: 327,634 (1.25 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [22]:
# ──────────────────────────────────────────────
# 6. Training loop
# ──────────────────────────────────────────────

EPOCHS = 500
start_epoch = 1
best_acc = 0.0


epoch_list  = []
error_list  = []

# Custom training loop to replicate per-epoch logging
for epoch in range(start_epoch, start_epoch + EPOCHS):
    print(f'\nEpoch: {epoch}')

    # --- Train ---
    train_results = model.fit(
        train_dataset,
        epochs=1,
        verbose=1,
        callbacks=[lr_callback],
    )

    # --- Test ---
    test_results = model.evaluate(test_dataset, verbose=0)
    test_loss    = test_results[0]
    test_acc     = test_results[1]
    test_error   = 1.0 - test_acc

    print(f'test: {test_acc:.4f} || {test_error:.4f}')

    epoch_list.append(epoch)
    error_list.append(test_error)

    if test_acc > best_acc:
        best_acc = test_acc
        model.save_weights('best_sknet_weights.h5')
        print(f'  >> New best accuracy: {best_acc:.4f}  (weights saved)')

        print(f'\nTraining complete. Best test accuracy: {best_acc:.4f}')



Epoch: 1

Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.


2026-02-18 16:27:34.535717: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 67756032 exceeds 10% of free system memory.


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 6)

## The Method in replicating the MTHAR work 
To reconstruct the continuous time-series stream from the pre-segmented UCI HAR dataset according to the methodology in the Duan et al. (2023) paper, you need to "stitch" the overlapping windows back together.

The UCI HAR dataset was originally segmented into windows of 128 samples with a 50% overlap (64 samples)2. Because the paper focuses on activity segmentation (identifying starting and ending positions), reconstructing the full stream is necessary to define the ground truth boundaries $(t^x, l^x)$ for their multi-task learning model

1. Grouping by Subject
The first step is to isolate data for each individual participant.Use the subject_train.txt file, which contains a subject ID (1–30) for every row (window) in your feature matrix.Filter your raw signal tensor—shaped (7352, 128, 9)—so that you only process windows belonging to one subject at a time.

2. The Reconstruction (Stitching) Process
Since each window starts 64 samples after the previous one, you must blend the overlapping regions. The paper implies a "smoothing" approach through averaging to handle the transitions between windows.

Logic for one channel of a single subject:Initialize: Create an empty array for the reconstructed signal and a "counter" array of the same length to track how many windows cover each index.
Total Length = (Number of Windows * 64) + 648.
Accumulate: Loop through each window $i$:Place the 128 samples into the reconstructed array starting at index $i \times 64$.
Add 1 to the corresponding indices in your counter array.
Average: Divide the total accumulated signal by the counter array.Indices 0–63 and the final 64 samples will be divided by 1.Indices 64 to (Total Length - 64) will be divided by 2 (because they represent the overlap)

3. Reconstructing Ground Truth Labels
The y_train file provides one label per window. To replicate the researchers' "Ground Truth Boundaries," you must map these back to the samples:Assign Labels: 
Assign the label of Window $i$ to the 64-sample "step" starting at $i \times 64$.
Define Boundaries: Scan the resulting sample-level label sequence.
 A boundary is identified whenever the activity label changes (e.g., from WALKING to SITTING).Center and Length $(t^x, t^l)$: Convert these segments into the paper's required format:$t^x$: The center coordinate of the activity segment15.$t^l$: The total number of samples (length) of that activity
 
 4. Summary for the 9 ChannelsRepeat this process for all 9 channels (X/Y/Z for Total Acc, Body Acc, and Gyro). 
 You will end up with a continuous multivariate stream of shape (Total_Samples, 9) for each subject, which the MTHARS model uses to predict both the activity class and the offset to the true bound


In [ ]:

def reconstruct_har_data(X_windows, y_windows, subject_ids):
    """
    Reconstructs continuous signals and activity boundaries per subject.
    X_windows: (7352, 128, 9) raw signal tensor
    y_windows: (7352,) activity labels
    subject_ids: (7352,) subject IDs from subject_train.txt
    """
    unique_subjects = np.unique(subject_ids)
    all_reconstructed_data = {}

    for sub in unique_subjects:
        # 1. Filter data for the specific subject
        # Use .reshape(-1) to ensure the mask is a flat 1D array
        mask = (subject_ids.reshape(-1) == sub) 

# Explicitly filter the first axis
        sub_X = X_windows[mask, :, :]  
        sub_y = y_windows[mask]
        
        num_windows = sub_X.shape[0]
        overlap = 64 # 50% overlap of 128 samples
        total_len = (num_windows * overlap) + overlap
        
        # 2. Initialize reconstructed signal and count array for averaging
        sub_stream = np.zeros((total_len, 9))
        counts = np.zeros((total_len, 1))
        
        # 3. Stitch windows with 50% overlap
        for i in range(num_windows):
            start = i * overlap
            end = start + 128
            sub_stream[start:end, :] += sub_X[i]
            counts[start:end] += 1
            
        # 4. Average overlapping sections to smooth the signal
        sub_stream /= counts
        
        # 5. Reconstruct label sequence (propagate window label to its 64-sample step)
        sub_labels = np.zeros(total_len)
        for i in range(num_windows):
            sub_labels[i*overlap : (i+1)*overlap] = sub_y[i]
        # Fill the final 64 samples with the last known label
        sub_labels[-64:] = sub_y[-1]
        
        # 6. Extract Boundaries (tx: center, tl: length) as per Duan et al.
        boundaries = []
        # Find indices where labels change
        change_points = np.where(np.diff(sub_labels) != 0)[0]
        start_idx = 0
        
        for cp in change_points:
            end_idx = cp
            length = end_idx - start_idx + 1
            center = start_idx + (length / 2)
            boundaries.append({'activity': sub_labels[start_idx], 'tx': center, 'tl': length})
            start_idx = end_idx + 1
            
        all_reconstructed_data[sub] = {
            'signal': sub_stream, 
            'labels': sub_labels,
            'boundaries': boundaries
        }
        
    return all_reconstructed_data

# --- Usage Example ---
# Assuming X_train_raw is (7352, 128, 9)
# y_train is (7352,)
# sub_train is (7352,) from subject_train.txt
# reconstructed = reconstruct_har_data(X_train_raw, y_train, sub_train)

In [7]:
y_train  = np.loadtxt("/home/pschye/Documents/KCCR - Biosignals/Human Activity Recognition/Human Activity Recognition/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt")
subjects = np.loadtxt("/home/pschye/Documents/KCCR - Biosignals/Human Activity Recognition/Human Activity Recognition/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/subject_train.txt")

In [8]:
subjects

array([ 1.,  1.,  1., ..., 30., 30., 30.], shape=(7352,))

In [ ]:
# Apply the function to your data
reconstructed_data = reconstruct_har_data(X_train, y_train, subjects)

# Example: Accessing the continuous data for Subject 3
sub1_signal = reconstructed_data[3]['signal']   # Continuous (Length, 9) stream
sub1_labels = reconstructed_data[3]['labels']   # Sample-level activity labels
sub1_events = reconstructed_data[3]['boundaries'] # List of (tx, tl) events

In [ ]:
print(len(sub1_signal))

21888


# MTHAR - THE METHOD IN ACTION USING TENSORFLOW


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# ==========================================
# 1. SELECTIVE KERNEL (Backbone Integrity)
# ==========================================
class SKConv(layers.Layer):
    def __init__(self, features, M=3, G=32, r=32, stride=1, L=32):
        super(SKConv, self).__init__()
        self.M = M
        self.features = features
        self.d = max(int(features / r), L)
        
        self.convs = []
        for i in range(M):
            self.convs.append(models.Sequential([
                layers.Conv2D(features, kernel_size=(3, 1), strides=stride, 
                              padding='same', dilation_rate=(1 + i, 1), groups=G, use_bias=False),
                layers.BatchNormalization(),
                layers.ReLU()
            ]))

        self.gap = layers.GlobalAveragePooling2D()
        self.fc = models.Sequential([
            layers.Dense(self.d, use_bias=False),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.fcs = [layers.Dense(features) for _ in range(M)]
        self.softmax = layers.Softmax(axis=1)

    def call(self, x):
        feats = [conv(x) for conv in self.convs]
        feats_stack = tf.stack(feats, axis=1)
        
        feats_U = tf.reduce_sum(feats_stack, axis=1)
        feats_S = self.gap(feats_U)
        feats_Z = self.fc(feats_S)
        
        attention_vectors = [fc(feats_Z) for fc in self.fcs]
        attention_vectors = tf.stack(attention_vectors, axis=1)
        attention_vectors = self.softmax(attention_vectors)
        attention_vectors = tf.expand_dims(tf.expand_dims(attention_vectors, axis=2), axis=2)
        
        return tf.reduce_sum(feats_stack * attention_vectors, axis=1)

# ==========================================
# 2. PHASE 2: MULTI-SCALE WINDOW LOGIC
# ==========================================
SCALES = [0.2, 0.4, 0.6, 0.8, 1.0]
NUM_ANCHORS_PER_POS = len(SCALES) * 2 # w1 and w2 for each scale

# ==========================================
# 3. PHASE 3: MULTI-TASK MODEL (MTHARS)
# ==========================================
def build_mthars_model(input_shape=(128, 9, 1), num_classes=6):
    inputs = layers.Input(shape=input_shape)

    # --- Backbone (SKNet) ---
    x = layers.Conv2D(64, (5, 1), strides=(2, 1), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = SKConv(128)(x)
    x = SKConv(256)(x) # Feature Map output

    # --- Multi-Task Heads (Detection Style) ---
    # Recognition Head: Predicts class for every anchor at every time step
    # Shape: [Batch, Time_Steps, 1, Anchors * Classes]
    cls_output = layers.Conv2D(NUM_ANCHORS_PER_POS * num_classes, (3, 1), padding='same', name='cls_head')(x)
    
    # Localization Head: Predicts (fx, fl) offsets for every anchor
    # Shape: [Batch, Time_Steps, 1, Anchors * 2]
    loc_output = layers.Conv2D(NUM_ANCHORS_PER_POS * 2, (3, 1), padding='same', name='loc_head')(x)

    # Reshaping for Loss Calculation
    # We flatten the spatial dimensions to treat all anchors across the sequence as one batch
    cls_output = layers.Reshape((-1, num_classes), name='recognition')(cls_output)
    loc_output = layers.Reshape((-1, 2), name='localization')(loc_output)

    return models.Model(inputs=inputs, outputs=[cls_output, loc_output])

# ==========================================
# 4. TRAINING CRITERIA (Multi-Task Loss)
# ==========================================
def mthars_loss(y_true_cls, y_pred_cls, y_true_loc, y_pred_loc):
    # Classification: Categorical Cross Entropy (Recognition)
    cls_loss = tf.keras.losses.CategoricalCrossentropy()(y_true_cls, y_pred_cls)
    
    # Regression: Smooth L1 Loss (Localization/Offsets)
    # Only calculate localization loss for "Positive" anchors (not background)
    loc_loss = tf.keras.losses.Huber()(y_true_loc, y_pred_loc)
    
    return cls_loss + loc_loss # Total Multi-task Loss

2026-01-29 22:06:55.975363: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-29 22:07:02.823997: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-29 22:07:18.657722: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [ ]:
# The Selective Kernel Convolution (SKConv) is a key component of the backbone network in the MTHARS architecture.
#  It allows the model to adaptively select features from multiple convolutional branches with different receptive fields,
#  enhancing the model's ability to capture multi-scale temporal patterns in the input signals. 
# The SKConv layer consists of multiple convolutional branches, a global average pooling layer for feature fusion,
#  and attention mechanisms to weigh the importance of each branch's output before combining them into a single feature map for further processing in the network.
class SKConv(layers.Layer):
    def __init__(self, features, M=3, G=32, r=32, stride=1, L=32):
        super(SKConv, self).__init__()
        self.M = M
        self.features = features
        self.d = max(int(features / r), L)
        
        # Branching Convolutions (Split)
        self.convs = []
        for i in range(M):
            # Using dilation to mimic different receptive fields as per the paper
            self.convs.append(models.Sequential([
                layers.Conv2D(features, kernel_size=(3, 1), strides=stride, 
                              padding='same', dilation_rate=(1 + i, 1), groups=G, use_bias=False),
                layers.BatchNormalization(),
                layers.ReLU()
            ]))

        self.gap = layers.GlobalAveragePooling2D()
        
        # Attention bottleneck (Fuse)
        self.fc = models.Sequential([
            layers.Dense(self.d, use_bias=False),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        
        # Selection heads
        self.fcs = [layers.Dense(features) for _ in range(M)]
        self.softmax = layers.Softmax(axis=1)

    def call(self, x):
        batch_size = tf.shape(x)[0]
        
        # Split
        feats = [conv(x) for conv in self.convs] # List of [batch, h, w, c]
        feats_stack = tf.stack(feats, axis=1)    # [batch, M, h, w, c]
        
        # Fuse
        feats_U = tf.reduce_sum(feats_stack, axis=1)
        feats_S = self.gap(feats_U)
        feats_Z = self.fc(feats_S)
        
        # Select
        attention_vectors = [fc(feats_Z) for fc in self.fcs]
        attention_vectors = tf.stack(attention_vectors, axis=1) # [batch, M, c]
        attention_vectors = self.softmax(attention_vectors)
        
        # Reshape for broadcasting: [batch, M, 1, 1, c]
        attention_vectors = tf.expand_dims(tf.expand_dims(attention_vectors, axis=2), axis=2)
        
        # Weighted sum
        feats_V = tf.reduce_sum(feats_stack * attention_vectors, axis=1)
        return feats_V

def build_mthars_model(input_shape=(128, 9, 1), num_classes=6):
    inputs = layers.Input(shape=input_shape)

    # --- Backbone Network ---
    # Initial Conv Block
    x = layers.Conv2D(64, (5, 1), strides=(3, 1), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # SKConv Blocks
    x = SKConv(128)(x)
    x = SKConv(256)(x)
    
    # Flatten/Dense (Transition to Recognition/Segmentation Network)
    # The paper uses a "Windows Generate" and Conv1D structure here
    # For a direct conversion of your snippet:
    x_flatten = layers.Flatten()(x)
    
    # --- Multi-Task Heads (Based on Paper Figure 3) ---
    
    # 1. Recognition Head (Activity Category)
    rec_head = layers.Dense(num_classes, name='recognition_output')(x_flatten)
    rec_head = layers.LayerNormalization()(rec_head)
    rec_output = layers.Activation('softmax')(rec_head)
    
    # 2. Segmentation Head (Activity vs Transition)
    seg_head = layers.Dense(2, name='segmentation_output')(x_flatten)
    seg_head = layers.LayerNormalization()(seg_head)
    seg_output = layers.Activation('softmax')(seg_head)

    model = models.Model(inputs=inputs, outputs=[rec_output, seg_output])
    return model

# Initialize and Compile
model = build_mthars_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'recognition_output': 'categorical_crossentropy',
        'segmentation_output': 'categorical_crossentropy'
    },
    metrics=['accuracy']
)

model.summary()

2026-01-29 18:47:32.575002: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-29 18:47:43.325763: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-29 18:48:30.245793: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-29 18:48:52.999769: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 9, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 43, 9, 64) │        384 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 43, 9, 64) │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 43, 9, 64) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sk_conv (SKConv)    │ (None, 43, 9,     │     20,736 │ re_lu[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sk_conv_1 (SKConv)  │ (None, 43, 9,     │     45,952 │ sk_conv[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 99072)     │          0 │ sk_conv_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ recognition_output  │ (None, 6)         │    594,438 │ flatten[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ segmentation_output │ (None, 2)         │    198,146 │ flatten[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 6)         │         12 │ recognition_outp… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 2)         │          4 │ segmentation_out… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 6)         │          0 │ layer_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 2)         │          0 │ layer_normalizat… │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 859,928 (3.28 MB)

 Trainable params: 857,368 (3.27 MB)

 Non-trainable params: 2,560 (10.00 KB)